주피터 SQL 엔진 설정

In [ ]:
!pip install ipython-sql==0.4.1
!pip install SQLAlchemy==1.4.49

In [2]:
%load_ext sql

월별 세션수를 계산하는 SQL

In [ ]:
%%sql

SELECT
    LEFT(ts, 7) AS mon,
    COUNT(1) AS session_count
FROM raw_data.session_timestamp
GROUP BY 1
ORDER BY 1;

가장 많이 사용된 채널은 무엇인가?

In [ ]:
%%sql

SELECT
    channel,
    COUNT(1) AS session_count,
    COUNT(DISTINCT userId) AS user_count
FROM raw_data.user_session_channel
GROUP BY 1                         -- GROUP BY channel
ORDER BY 2 DESC;              -- ORDER BY session_count DESC

In [ ]:
%%sql

SELECT
    channel,
    COUNT(1) AS session_count,
    COUNT(DISTINCT userId) AS user_count
FROM raw_data.user_session_channel
GROUP BY 1                         -- GROUP BY channel
ORDER BY 3 DESC;              -- ORDER BY user_count DESC

가장 많은 세션을 만들어낸 사용자 ID는 무엇인가?

In [ ]:
%%sql

SELECT
    userId,
    COUNT(1) AS count
FROM raw_data.user_session_channel
GROUP BY 1                         -- GROUP BY userId
ORDER BY 2 DESC              -- ORDER BY count DESC
LIMIT 1;

월별 유니크한 사용자 수

In [ ]:
%%sql

SELECT
  TO_CHAR(A.ts, 'YYYY-MM') AS month,
  COUNT(DISTINCT B.userid) AS mau
FROM raw_data.session_timestamp A
JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1
ORDER BY 1 DESC;

In [ ]:
%%sql

SELECT
  TO_CHAR(A.ts, 'YYYY-MM') AS month,
  COUNT(B.userid) AS cnt,
  COUNT(DISTINCT B.userid) AS mau
FROM raw_data.session_timestamp A
JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1
ORDER BY 1 DESC;

월별 채널별 유니크한 사용자 수

In [ ]:
%%sql

SELECT

  TO_CHAR(A.ts, 'YYYY-MM') AS month,
  channel,
  COUNT(DISTINCT B.userid) AS mau
FROM raw_data.session_timestamp A
JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1, 2
ORDER BY 1 DESC, 2;

### CTAS & CTE

CTAS: SELECT를 가지고 테이블 생성

In [ ]:
CTAS: SELECT를 가지고 테이블 생성

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.yeojun_session_summary;
CREATE TABLE adhoc.yeojun_session_summary AS
SELECT B.*, A.ts FROM raw_data.session_timestamp A
JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid;

In [ ]:
%%sql

SELECT
  TO_CHAR(ts, 'YYYY-MM') AS month,
  COUNT(DISTINCT userid) AS mau
FROM adhoc.yeojun_session_summary
GROUP BY 1
ORDER BY 1 DESC;

중복된 레코드들 체크하기

In [ ]:
%%sql

SELECT COUNT(1) FROM adhoc.yeojun_session_summary;

In [ ]:
%%sql

SELECT COUNT(1)
FROM (
    SELECT DISTINCT userId, sessionId, ts, channel FROM adhoc.yeojun_session_summary
);

In [ ]:
%%sql

With ds AS (
  SELECT DISTINCT userId, sessionId, ts, channel FROM adhoc.yeojun_session_summary
)
SELECT COUNT(1)
FROM ds;

최근 데이터의 존재 여부 체크하기 (freshness)

In [ ]:
%%sql

SELECT MIN(ts), MAX(ts)
FROM adhoc.yeojun_session_summary;

Primary key uniqueness가 지켜지는지 체크하기

In [ ]:
%%sql

SELECT sessionId, COUNT(1)
FROM adhoc.yeojun_session_summary
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;


값이 비어있는 컬럼들이 있는지 체크하기

In [ ]:
%%sql

SELECT
    COUNT(CASE WHEN sessionId is NULL THEN 1 END) sessionid_null_count,
    COUNT(CASE WHEN userId is NULL THEN 1 END) userid_null_count,
    COUNT(CASE WHEN ts is NULL THEN 1 END) ts_null_count,
    COUNT(CASE WHEN channel is NULL THEN 1 END) channel_null_count
FROM adhoc.yeojun_session_summary;

2개의 새 테이블 소개
- 지금까지 session_timestamp, user_sesion_channel
- 추가 raw_data.session_transaction, raw_data,channel

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.yeojun_channel_month_revenue;
CREATE TABLE adhoc.yeojun_channel_month_revenue AS
SELECT
  TO_CHAR(ts,'YYYY-MM') as month,
  channel,
  COUNT(DISTINCT userid) uniqueUsers,
  COUNT(DISTINCT CASE WHEN amount > 0 THEN userid END) paidUsers,
  ROUND(paidUsers*100.0/uniqueUsers,2) as conversionRate,
  SUM(B.amount) grossRevenue,
  SUM(CASE WHEN refunded ='False' THEN amount END) netRevenue
FROM adhoc.yeojun_session_summary A
LEFT JOIN raw_data.session_transaction B ON A.sessionid = B.sessionid
GROUP BY month, channel;

In [ ]:
%%sql

SELECT *
FROM adhoc.yeojun_channel_month_revenue;